<a href="https://colab.research.google.com/github/tommyA8/wangchanBERTa-fined-tune-thaiqa_squad/blob/main/wangchanBERTa_fined_tune_thaiqa_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation and Importing library

In [ ]:
# !pip install -q transformers[sentencepiece]
!pip install -q transformers[torch]
!pip install -q datasets
!pip -q install bs4

from pandas import DataFrame
from bs4 import BeautifulSoup
from datasets import load_dataset, Dataset, DatasetDict

#load_dataset

In [ ]:
thaiqa_dataset = load_dataset("thaiqa_squad")
thaiqa_dataset

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})

# Example for Prepare-Data

##1.Data Cleaning

In [ ]:
# example = dataset['train'][0]['context']
# example

#### Step 1: HTML Tags

In [ ]:
# from bs4 import BeautifulSoup
# # remove HTML tag
# example = BeautifulSoup(example).get_text()
# example

#### Step 2: Punctuation

In [ ]:
# import string
# punct = string.punctuation

# # remove these punctuation
# no_punct = [char for char in example if char not in punct]
# words_wo_punct = ''.join(no_punct)
# words_wo_punct

### Assemble it

In [ ]:
from bs4 import BeautifulSoup
import string
punct = string.punctuation
# print(punct)

def cleansing_data(example):
  example = BeautifulSoup(example).get_text() # Removal HTML Tag
  # no_punct = [char for char in example if char not in punct] # Removal Punctuation #----> ไม่ลบแล้ว เพราะตัวคำตอบใน dataset มันมี
  # example  = ''.join(no_punct)
  return example

In [ ]:
# dataset_train_df['clean_context'] = dataset_train_df['context'].apply(lambda x: cleansing_data(x))
# # dataset_valid_df['clean_context'] = dataset_valid_df['context'].apply(lambda x: cleansing_data(x))

# # dataset_train_df.head(10)
# # dataset_valid_df

##2.Find Answer-start and Answer-end

In [ ]:
#Due to cleaning data process, The length of data is changed

In [ ]:
# #Find answer start and answer end
# def find_new_answer(example_clean_context, example_answers):
#   cntx = example_clean_context
#   ans = example_answers
#   answer_start = cntx.index(ans['answer'][0])
#   answer_end = answer_start + len(ans['answer'][0])
#   if ans['answer'][0] == cntx[answer_start:answer_end]:
#     ans['answer_begin_position'][0] = int(answer_start)
#     ans['answer_end_position'][0] = int(answer_end)
#     return ans
#   else:
#     return "No answer contain in Context"

In [ ]:
# dataset_train_df['new_answers'] = dataset_train_df.apply(lambda x: find_new_answer(x['clean_context'], x['answers']), axis=1 )
# # dataset_valid_df['new_answers'] = dataset_train_df.apply(lambda x: find_new_answer(x['clean_context'], x['answers']), axis=1 )

# # dataset_train_df.head(10)
# # dataset_valid_df

# Data-preparation Function (thaiqa_dataset)

In [ ]:
thaiqa_train_df = DataFrame(thaiqa_dataset['train'])
thaiqa_valid_df = DataFrame(thaiqa_dataset['validation'])

In [ ]:
thaiqa_train_df.head(4)

,question_id,article_id,context,question,answers
0,1,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"{'answer': ['ฮิกกิ้นส์'], 'answer_begin_positi..."
1,2035,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",เพลง Benji's Theme ใช้ประกอบภาพยนตร์เรื่อง Ben...,"{'answer': ['ชาร์ลี ริช'], 'answer_begin_posit..."
2,2,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['เมชตา'], 'answer_begin_position':..."
3,3,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['ลมสุริยะ'], 'answer_begin_positio..."


In [ ]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punct = [char for char in punct if not ( char == ','  or char == '"' or char == '.' or char == '/' or char == "'" ) ]  #ลองไม่ลบ " , " กับ " ' " และ " . " ว่าจะเอา "/" ด้วย
punct  = ''.join(punct)
punct

'!#$%&()*+-:;<=>?@[\\]^_`{|}~'

In [ ]:
# 1.cleaning -> 2. find new_answer position ->  3. lowercase for WangchanBERTa Model
def prepare_data(example_context, example_question, example_answers):
      #cleaning_context
      example_context = BeautifulSoup(example_context).get_text() # Removal HTML Tag

      #lowercase
      example_context = example_context.lower()
      example_question = example_question.lower()
      example_answers['answer'][0] = example_answers['answer'][0].lower()

      # Removal Punctuation
      no_punct_context = [char for char in example_context if char not in punct] #context
      example_context  = ''.join(no_punct_context)

      no_punct_question = [char for char in example_question if char not in punct] #question
      example_question  = ''.join(no_punct_question)

      no_punct_answers = [char for char in example_answers['answer'][0] if char not in punct]
      example_answers['answer'][0]  = ''.join(no_punct_answers)

      #find new answer position
      answer_start = example_context.index(example_answers['answer'][0])
      answer_end = answer_start + len(example_answers['answer'][0])

      if example_answers['answer'][0] == example_context[answer_start:answer_end]:
          example_answers['answer_begin_position'][0] = answer_start
          example_answers['answer_end_position'][0] = answer_end
          return example_context, example_question, example_answers
      else:
          return "No answer contain in Context"

In [ ]:
thaiqa_train_df['clean_context'], thaiqa_train_df['new_question'], thaiqa_train_df['new_answers'] = zip(*thaiqa_train_df.apply(
    lambda x: prepare_data(x['context'], x['question'], x['answers']), axis=1 ))

thaiqa_valid_df['clean_context'], thaiqa_valid_df['new_question'], thaiqa_valid_df['new_answers'] = zip(*thaiqa_valid_df.apply(
    lambda x: prepare_data(x['context'], x['question'], x['answers']), axis=1 ))

In [ ]:
thaiqa_train_df.head(4)

,question_id,article_id,context,question,answers,clean_context,new_question,new_answers
0,1,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,"{'answer': ['ฮิกกิ้นส์'], 'answer_begin_positi...",เบนจี้ เบนจี้ เป็นชื่อตัวละครหมาพันทางแสนรู้ ...,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง ben...,"{'answer': ['ฮิกกิ้นส์'], 'answer_begin_positi..."
1,2035,115035,"<doc id=""115035"" url=""https://th.wikipedia.org...",เพลง Benji's Theme ใช้ประกอบภาพยนตร์เรื่อง Ben...,"{'answer': ['ชาร์ลี ริช'], 'answer_begin_posit...",เบนจี้ เบนจี้ เป็นชื่อตัวละครหมาพันทางแสนรู้ ...,เพลง benji's theme ใช้ประกอบภาพยนตร์เรื่อง ben...,"{'answer': ['ชาร์ลี ริช'], 'answer_begin_posit..."
2,2,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['เมชตา'], 'answer_begin_position':...",ลูนา 1 ลูนา 1 อี1 ซีรีส์ ซึ่งในขณะนั้นรู้จักกั...,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['เมชตา'], 'answer_begin_position':..."
3,3,376583,"<doc id=""376583"" url=""https://th.wikipedia.org...",ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['ลมสุริยะ'], 'answer_begin_positio...",ลูนา 1 ลูนา 1 อี1 ซีรีส์ ซึ่งในขณะนั้นรู้จักกั...,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,"{'answer': ['ลมสุริยะ'], 'answer_begin_positio..."


In [ ]:
thaiqa_valid_df.head(4)

,question_id,article_id,context,question,answers,clean_context,new_question,new_answers
0,3,639294,"<doc id=""639294"" url=""https://th.wikipedia.org...",เฝิง เส้าเฟิง รับบทอะไรใน The Palace,"{'answer': ['องค์ชาย 8'], 'answer_begin_positi...",เฝิง เส้าเฟิง เฝิง เส้าเฟิง หรือ วิลเลี่ยม เฝิ...,เฝิง เส้าเฟิง รับบทอะไรใน the palace,"{'answer': ['องค์ชาย 8'], 'answer_begin_positi..."
1,5,135329,"<doc id=""135329"" url=""https://th.wikipedia.org...",ชิงชัน เป็นพืชชนิดใด,"{'answer': ['ไม้ยืนต้น'], 'answer_begin_positi...",ชิงชัน ชิงชัน เป็นชื่อของไม้ยืนต้นขนาดกลางจนถึ...,ชิงชัน เป็นพืชชนิดใด,"{'answer': ['ไม้ยืนต้น'], 'answer_begin_positi..."
2,6,754945,"<doc id=""754945"" url=""https://th.wikipedia.org...",แฝดไข่ร่วมคืออะไร,{'answer': ['ลูกทั้งสองเจริญมาจากไซโกตเดียวแล้...,แฝด แฝด คือ ลูกสองคนหรือตัวที่เกิดในการตั้งคร...,แฝดไข่ร่วมคืออะไร,{'answer': ['ลูกทั้งสองเจริญมาจากไซโกตเดียวแล้...
3,8,524853,"<doc id=""524853"" url=""https://th.wikipedia.org...",สมเด็จพระราชินีนาถพระองค์เดียวของสเปนมีพระนามว...,{'answer': ['สมเด็จพระราชินีนาถอีซาเบลที่ 2 แห...,สมเด็จพระราชินีนาถอีซาเบลที่ 2 แห่งสเปน สมเด็จ...,สมเด็จพระราชินีนาถพระองค์เดียวของสเปนมีพระนามว...,{'answer': ['สมเด็จพระราชินีนาถอีซาเบลที่ 2 แห...


#Convert Dataframe to Huggingface datasets

In [ ]:
# create dataframe (thaiqa)
thaiqa_train = DataFrame()
thaiqa_train['question_id'] = thaiqa_train_df['question_id']
thaiqa_train['article_id'] = thaiqa_train_df['article_id']
thaiqa_train['context'] = thaiqa_train_df['clean_context']
thaiqa_train['question'] = thaiqa_train_df['new_question']
thaiqa_train['answers'] = thaiqa_train_df['new_answers']
thaiqa_valid = DataFrame()
thaiqa_valid['question_id'] = thaiqa_valid_df['question_id']
thaiqa_valid['article_id'] = thaiqa_valid_df['article_id']
thaiqa_valid['context'] = thaiqa_valid_df['clean_context']
thaiqa_valid['question'] = thaiqa_valid_df['new_question']
thaiqa_valid['answers'] = thaiqa_valid_df['new_answers']

In [ ]:
import datasets

thaiqa_train = Dataset.from_dict(thaiqa_train)
thaiqa_valid = Dataset.from_dict(thaiqa_valid)
thaiqa_dataset = datasets.DatasetDict({"train": thaiqa_train,"validation": thaiqa_valid})

In [ ]:
thaiqa_dataset['train'], thaiqa_dataset['validation']

(Dataset({
     features: ['question_id', 'article_id', 'context', 'question', 'answers'],
     num_rows: 4000
 }),
 Dataset({
     features: ['question_id', 'article_id', 'context', 'question', 'answers'],
     num_rows: 74
 }))

#Example for Pre-processing

**WangchanBERta's paper https://arxiv.org/pdf/2101.09635.pdf**

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #using the same tokenizer of WangchanBERTa

## Each example from Datset

In [ ]:
# # example = thaiqa_dataset['train'][0]
# # example
# example = iapp_dataset['train'][71]
# example

In [ ]:
# len(tokenizer(example["question"], example["context"])["input_ids"])

In [ ]:
# max_length = 416 #384 # The maximum length of a feature (question and context)
# doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
# inputs = tokenizer(
#     example['question'],
#     example['context'],
#     truncation="only_second", #Truncation only context
#     max_length=max_length,
#     stride=doc_stride, #overlapping of overflowing tokens
#     return_overflowing_tokens=True, #return multiple input ids if exceeding max_length
#     return_offsets_mapping=True,
#     padding="max_length",
# )

In [ ]:
# inputs.keys()

In [ ]:
# print(len(inputs["input_ids"]) )# Have 1 feature due to truncate
# print(len(inputs["input_ids"][0]))#Feature 1 has 384 token due to max_length
# print(inputs["input_ids"][0])
# # print(len(inputs["input_ids"]) )# Have 5 feature due to truncate
# # print(len(inputs["input_ids"][0]))#Feature 1 has 384 token due to max_length
# # print(len(inputs["input_ids"][-1]) )#Feature 5 has 384 token due to padding="max_length"

In [ ]:
# # จำนวนเลข 0 คือจำนวนฟีเจอร์
# print(inputs['overflow_to_sample_mapping'])

In [ ]:
# #ทดลอง Decoding
# print(tokenizer.decode(inputs['input_ids'][0]))
# #offset_mapping of feature 5
# print(inputs['offset_mapping'][0]) # offset_mapping ใช้บอกว่า token นั้น มี char ตั้งแต่ตำแหน่งเท่าไหร่ ถึงเท่าไหร่

In [ ]:
# sequence_ids = inputs.sequence_ids(-1)
# print(sequence_ids)
# #0 คือ Token ของ question
# #1 คือ Token ของ context

In [ ]:
# answers = example["answers"]
# start_char = answers["answer_start"][0] #ตำแหน่ง character ตัวแรกของคำตอบ
# end_char = answers["answer_end"][0] #ตำแหน่ง character ตัวสุดท้ายของคำตอบ

# #ใช้หา token ของ context ที่ span อยู่ใน input_ids ดังตัวอย่างด้านบน
# token_start_index = 0
# while sequence_ids[token_start_index] != 1:
#     token_start_index += 1 # Start token index of the current span in the text.

# token_end_index = len(inputs["input_ids"][0]) - 1 #384 - 1 => นับ 0-383 เท่่ากับ 384 token
# while sequence_ids[token_end_index] != 1:
#     token_end_index -= 1 # End token index of the current span in the text.

# offsets = inputs["offset_mapping"][0]

# # """ตรวจสอบว่า Answer อยู่ใน span ของ context ที่ฟีเจอร์นี้หรือไม่ """
# if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
#   #running from 25 to 440
#   while offsets[token_start_index][0] <= start_char: #440
#     token_start_index += 1
#   answer_start_pos_token = token_start_index - 1 #140

#   while offsets[token_end_index][1] >= end_char: #449
#     token_end_index -= 1
#   answer_end_pos_token = token_end_index + 1 #140

#   print("Start_End Position: ",answer_start_pos_token,  answer_end_pos_token)
# else:
#   print("The answer is not in this feature.")

# # inputs["start_positions"].append(answer_start_pos_token)
# # inputs["end_positions"].append(answer_end_pos_token)

# print(tokenizer.decode(inputs["input_ids"][0][answer_start_pos_token:answer_end_pos_token+1]))
# print(answers["answer"][0])

# assert answers["answer"][0] == tokenizer.decode(inputs["input_ids"][0][answer_start_pos_token:answer_end_pos_token+1]), "Answer is not correct"

In [ ]:
# print("Answer is: ",tokenizer.decode(inputs["input_ids"][0][147]))
# print("Answer is: ",tokenizer.decode(inputs["input_ids"][0][148]))
# print("Answer is: ",tokenizer.decode(inputs["input_ids"][0][149]))
# print("Answer is: ",tokenizer.decode(inputs["input_ids"][0][150]))

## Batch Example frome Dataset

In [ ]:
# batched_example = thaiqa_dataset["train"][450:455]
# # batched_example = dataset["train"][0:5]

# batched_inputs = tokenizer(
#     batched_example["question"],
#     batched_example["context"],
#     truncation="only_second", #Truncation only context
#     max_length=max_length,
#     stride=doc_stride, #overlapping of overflowing tokens
#     return_overflowing_tokens=True, #return multiple input ids if exceeding max_length
#     return_offsets_mapping=True,
#     padding="max_length",
# )

# print(f"The 5 examples gave {len(batched_inputs['input_ids'])} features.")
# print(f"Here is where each comes from: {batched_inputs['overflow_to_sample_mapping']}.")

In [ ]:
# batched_inputs.keys()

In [ ]:
# start_positions = []
# end_positions = []

# sample_mapping = batched_inputs["overflow_to_sample_mapping"]# 'The N examples gave M features.'
# offset_mapping = batched_inputs["offset_mapping"]

# for i, offsets in enumerate(offset_mapping):

#     sample_index = sample_mapping[i]
#     answers = batched_example["answers"][sample_index]
#     start_char = answers["answer_begin_position"][0] #ตำแหน่ง character ตัวแรกของคำตอบ
#     end_char = answers["answer_end_position"][0] #ตำแหน่ง character ตัวสุดท้ายของคำตอบ

#     sequence_ids = batched_inputs.sequence_ids(i)

#     token_context_start = 0
#     while sequence_ids[token_context_start] != 1:
#         token_context_start += 1

#     token_context_end = len(batched_inputs["input_ids"][i]) - 1
#     while sequence_ids[token_context_end] != 1:
#         token_context_end -= 1

#     # if the answer is in the corresponding span of the context: start_context < answer position <end_context
#     if (offsets[token_context_start][0] <= start_char and offsets[token_context_end][1] >= end_char):
#         while offsets[token_context_start][0] <= start_char:
#             token_context_start += 1
#         token_context_start = token_context_start - 1

#         while offsets[token_context_end][1] >= end_char:
#             token_context_end -= 1
#         token_context_end = token_context_end + 1

#         try:
#             # detect incorrect answer when compare with real answer from dataset cause's 'answer_begin_position' and 'answer_end_position' is wrong
#             decode_answer = tokenizer.decode(batched_inputs["input_ids"][i][token_context_start:token_context_end+1])
#             assert decode_answer == answers["answer"][0]
#             start_positions.append(token_context_start)
#             end_positions.append(token_context_end)
#         except:
#             print('incorrect', answers["answer"][0], 'correct:', decode_answer)
#             start_positions.append(0)
#             end_positions.append(0)
#     else: # if the answer is not in the corresponding span of the context
#         start_positions.append(0)
#         end_positions.append(0)

# start_positions, end_positions

In [ ]:
# batched_inputs["overflow_to_sample_mapping"]

In [ ]:
# if len(batched_inputs["overflow_to_sample_mapping"]) == len(start_positions):
#   print(True)

In [ ]:
# idx = 3
# sample_index = batched_inputs["overflow_to_sample_mapping"][idx]
# answers = batched_example["answers"][sample_index]
# answer = answers["answer"][0]
# question = batched_example["question"][sample_index]

# start = start_positions[idx]
# end = end_positions[idx]
# labeled_answer = tokenizer.decode(batched_inputs["input_ids"][idx][start : end + 1])

# print(question)
# print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")
# print(f"start_position: {start}, end_positon: {end} ")

#Pre-processing (thaiqa_dataset)

**Ref. https://huggingface.co/course/chapter7/7?fw=pt#a-custom-training-loop**

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
list_wrong_answer_fromthaiqa_dataset = []

max_length = 416
doc_stride = 128

def pre_processing_thaiqa(examples):
  wrong_answer_fromthaiqa_dataset = {} # stored wrong answer
  inputs = tokenizer(
      examples['question'],
      examples['context'],
      truncation="only_second", # Truncation only context
      max_length=max_length,
      stride=doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )
  sample_mapping = inputs.pop("overflow_to_sample_mapping") # The N examples gave M features.
  offset_mapping = inputs.pop("offset_mapping")

  inputs["start_positions"] = []
  inputs["end_positions"] = []

  for i, offsets in enumerate(offset_mapping):

      sequence_ids = inputs.sequence_ids(i)

      token_context_start = 0
      while sequence_ids[token_context_start] != 1:
          token_context_start += 1

      token_context_end = len(inputs["input_ids"][i]) - 1
      while sequence_ids[token_context_end] != 1:
          token_context_end -= 1

      sample_index = sample_mapping[i]
      answers = examples["answers"][sample_index]
      start_char = answers["answer_begin_position"][0]
      end_char = answers["answer_end_position"][0]

      # if the answer is in the corresponding span of the context: (start_context <= answer position <= end_context)
      if (offsets[token_context_start][0] <= start_char and offsets[token_context_end][1] >= end_char):

          while offsets[token_context_start][0] <= start_char:
              token_context_start += 1
          start_positions = token_context_start - 1

          while offsets[token_context_end][1] >= end_char:
              token_context_end -= 1
          end_positions = token_context_end + 1

          try:
              # detect incorrect answer when compare with real answer from dataset cause's 'answer_begin_position' and 'answer_end_position' is wrong
              assert tokenizer.decode(inputs["input_ids"][i][start_positions:end_positions+1]) == answers["answer"][0]
              inputs["start_positions"].append(start_positions)
              inputs["end_positions"].append(end_positions)
          except:
              # print('incorrect',answers["answer"][0])
              wrong_answer_fromthaiqa_dataset['pre_process_ans'] = tokenizer.decode(inputs["input_ids"][i][start_positions:end_positions+1])
              wrong_answer_fromthaiqa_dataset['dataset_answer'] = answers['answer'][0]
              # wrong_answer_fromthaiqa_dataset['answers'] = examples['answers']
              list_wrong_answer_fromthaiqa_dataset.append(wrong_answer_fromthaiqa_dataset)
              wrong_answer_fromthaiqa_dataset = {}
              inputs["start_positions"].append(0)
              inputs["end_positions"].append(0)
      else:
          # if the answer is not in the corresponding span of the context
          inputs["start_positions"].append(0)
          inputs["end_positions"].append(0)

  return inputs

In [ ]:
example = pre_processing_thaiqa(thaiqa_dataset['train'][0:5])
print(len(example['start_positions']), len(example['input_ids']))
print(example['start_positions'])

5 5
[147, 136, 46, 150, 0]


In [ ]:
tokenized_thaiqa_dataset = thaiqa_dataset.map(pre_processing_thaiqa, batched=True, remove_columns=thaiqa_dataset["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [ ]:
len(thaiqa_dataset['train']), len(tokenized_thaiqa_dataset["train"])
#จำนวนเพิ่มขึ้นจากการ truncation ส่งผลให้มีฟีเจอร์เกิดขึ้นมาเพิ่ม

(4000, 16700)

In [ ]:
len(thaiqa_dataset["validation"]), len(tokenized_thaiqa_dataset['validation'])

(74, 268)

**'answer_begin_position' and 'answer_end_position' are incorrect.**

In [ ]:
len(list_wrong_answer_fromthaiqa_dataset)

576

In [ ]:
worng_ans_df = DataFrame(list_wrong_answer_fromthaiqa_dataset)
worng_ans_df

,pre_process_ans,dataset_answer
0,ในปี 1933,ปี 1933
1,ในปี 1933,ปี 1933
2,ของรัฐนาซี,รัฐนาซี
3,อดีตประธานาธิบดีวิลเลียม เอช. ทัฟต์,ประธานาธิบดีวิลเลียม เอช. ทัฟต์
4,เป็นอันดับที่ 2,อันดับที่ 2
...,...,...
571,ที่ดินดอนสามเหลี่ยมปากแม่น้ําอิรวดี,ดินดอนสามเหลี่ยมปากแม่น้ำอิรวดี
572,จาเร็ด คัชเนอร์,จาเร็ด คัชเนอร์
573,เมื่อเดือนตุลาคม ค.ศ. 1958,เดือนตุลาคม ค.ศ. 1958
574,ของประเทศฝรั่งเศส,ประเทศฝรั่งเศส


# Fine-tuning

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #using the same tokenizer of WangchanBERTa

Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 32
learning_rate = 2e-5

args = TrainingArguments(
    f"finetune_thaiqa_squad",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    warmup_ratio=0.2,
    weight_decay=0.01,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator = default_data_collator,
    train_dataset=tokenized_thaiqa_dataset['train'],
    eval_dataset=tokenized_thaiqa_dataset['validation'],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.631000,2.031461
2,1.660000,1.951821


TrainOutput(global_step=1044, training_loss=2.124743187564543, metrics={'train_runtime': 334.129, 'train_samples_per_second': 99.961, 'train_steps_per_second': 3.125, 'total_flos': 7090940735923200.0, 'train_loss': 2.124743187564543, 'epoch': 2.0})

In [ ]:
#----------------------------------------save model-------------
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

trainer.save_model("/content/drive/MyDrive/QA_finetune_thaiqa_squad")

#Using the fine-tuned model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model = "/content/drive/MyDrive/finetune_thaiqa_squad"
question_answerer = pipeline("question-answering", model=model)

context = """พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. 2533) หรือชื่อเล่นว่า อร เป็นนักแสดงหญิงชาวไทย สำเร็จมัธยมศึกษาจากCatholic Cathedral College ประเทศนิวซีแลนด์
และปริญญาตรีจากRaffles International College สาขา Business Marketing เข้าสู่วงการตั้งแต่อายุ 6 ขวบ จากการแสดงละครเวทีกับ ครูชลประคัลภ์ จันทร์เรือง
จากนั้นก็เล่นโฆษณาในวัยเด็ก 2- 3 ชิ้น และยังเคยแสดงช่วงละครสั้น ในรายการซุปเปอร์จิ๋ว ประมาณปี 2542 ปัจจุบันเป็นทั้ง นักแสดง ,
พิธีกร และ วีเจ อยู่ที่คลื่น เก็ท 102.5 Bangkok International Hits Music Station และยังเป็นพิธีกรให้กับช่อง ทรู มิวสิก"""

In [ ]:
question1 = "พัทธ์ธีราเข้าวงการตอนอายุเท่าไหร่"
question2 = "พัทธ์ธีราเข้าวงการตั้งแต่อายุเท่าไหร่"
question3 = "พัทธ์ธีราเข้าวงการกี่ขวบ"
question4 = "อายุที่พัทธ์ธีราเข้าวงการคือ"

ans1 = question_answerer(question=question1, context=context)
ans2 = question_answerer(question=question2, context=context)
ans3 = question_answerer(question=question3, context=context)
ans4 = question_answerer(question=question4, context=context)

DataFrame(
    {
        'question': [question1, question2, question3, question4],
        'answer': [ans1['answer'], ans2['answer'], ans3['answer'], ans4['answer'] ],
        'score': [ ans1['score'], ans2['score'], ans3['score'], ans4['score'] ]
    }
)

,question,answer,score
0,พัทธ์ธีราเข้าวงการตอนอายุเท่าไหร่,6 ขวบ,0.748101
1,พัทธ์ธีราเข้าวงการตั้งแต่อายุเท่าไหร่,6 ขวบ,0.589396
2,พัทธ์ธีราเข้าวงการกี่ขวบ,6 ขวบ,0.474511
3,อายุที่พัทธ์ธีราเข้าวงการคือ,6 ขวบ,0.714449


In [ ]:
question = "พัทธ์ธีรา ศรุติพงศ์โภคิน มีชื่อเล่นว่าอะไร"
question_answerer(question=question, context=context)

{'score': 0.9933181405067444, 'start': 67, 'end': 70, 'answer': ' อร'}

In [ ]:
question = "พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันไหน"
question_answerer(question=question, context=context)

{'score': 0.5565053224563599,
 'start': 30,
 'end': 51,
 'answer': ' 3 ธันวาคม พ.ศ. 2533)'}

In [ ]:
question = "พัทธ์ธีราสำเร็จมัธยมศึกษาจากไหน"
question_answerer(question=question, context=context)

{'score': 0.6329625248908997,
 'start': 92,
 'end': 138,
 'answer': ' สำเร็จมัธยมศึกษาจากCatholic Cathedral College'}

In [ ]:
context = """
ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการคือ เครือรัฐออสเตรเลีย (Commonwealth of Australia) เป็นประเทศซึ่งประกอบด้วยแผ่นดินหลักของทวีปออสเตรเลีย, เกาะแทสเมเนีย และเกาะอื่น ๆ ในมหาสมุทรอินเดีย แปซิฟิก และมหาสมุทรใต้ มันเป็นประเทศที่ใหญ่เป็นอันดับหกของโลกเมื่อนับพื้นที่ทั้งหมด ประเทศเพื่อนบ้านของออสเตรเลียประกอบด้วย อินโดนีเซีย ปาปัวนิวกินีและติมอร์-เลสเตทางเหนือ หมู่เกาะโซโลมอน วานูอาตู และนิวแคลิโดเนียทางตะวันออกเฉียงเหนือ และนิวซีแลนด์ทางตะวันออกเฉียงใต้ เป็นเวลาอย่างน้อย 40,000 ปี[6] ก่อนที่จะตั้งถิ่นฐานครั้งแรกของอังกฤษในศตวรรษที่ 18,[7][8] ประเทศออสเตรเลียเป็นที่อยู่อาศัยของชาวออสเตรเลียพื้นเมือง[9] ที่พูดภาษาที่แบ่งออกได้เป็นกลุ่มประมาณ 250 ภาษา.[10][11] หลังจากการค้นพบของทวีปโดยนักสำรวจชาวดัตช์ในปี ค.ศ. 1606, ครึ่งหนึ่งของฝั่งตะวันออกของออสเตรเลียถูกอ้างว่าเป็นของสหราชอาณาจักรในปี ค.ศ. 1770 และตั้งรกรากในขั้นต้นโดยการขนส่งนักโทษมายังอาณานิคมของนิวเซาธ์เวลส์จากวันที่ 26 มกราคม ค.ศ. 1788 จำนวนประชากรเพิ่มขึ้นอย่างต่อเนื่องในทศวรรษที่ผ่านมา ทวีปถูกสำรวจ และอีกห้าอาณานิคมปกครองตนเองของพระมหากษัตริย์ถูกจัดตั้งขึ้น เมื่อวันที่ 1 มกราคม ค.ศ. 1901 หกอาณานิคมถูกตั้งขึ้นเป็นสหพันธ์, รวมตัวกันเป็นเครือรัฐออสเตรเลีย. ตั้งแต่นั้นมา ออสเตรเลียยังคงรักษาระบบการเมืองเสรีนิยมประชาธิปไตยที่มั่นคง ที่ ทำหน้าที่เป็นรัฐสภาประชาธิปไตยของรัฐบาลกลางและพระมหากษัตริย์ตามรัฐธรรมนูญ สหพันธ์ประกอบด้วยหกรัฐ และอีกหลายพื้นที่ ประชากร 23.1 ล้านคน[12] อยู่ในเมืองเป็นส่วนใหญ่และมีความหนาแน่นอย่างมากในรัฐทางตะวันออก[13] ออสเตรเลียเป็นประเทศที่พัฒนาแล้ว และเป็นหนึ่งในประเทศที่ร่ำรวยที่สุดในโลกที่มีเศรษฐกิจ ใหญ่ที่สุดเป็นอันดับที่ 12 ของโลก ในปี ค.ศ. 2012 ออสเตรเลียมีรายได้ต่อหัวที่สูงที่สุดอันดับห้าของโลก[14] ค่าใช้จ่ายทางทหารของออสเตรเลียมากที่สุดเป็นอันดับที่ 13 ของโลก ด้วยดัชนีการพัฒนามนุษย์ที่สูงที่สุดอันดับที่สองทั่วโลก, ออสเตรเลียถูกจัดอันดับที่สูงในการเปรียบเทียบระหว่างประเทศจำนวนมากของประสิทธิภาพการทำงานในระดับชาติ เช่น คุณภาพชีวิต, สุขภาพ, การศึกษา, เสรีภาพทางเศรษฐกิจ, และการปกป้องเสรีภาพของพลเมืองและสิทธิทางการเมือง.[15] ออสเตรเลียเป็นสมาชิกของสหประชาชาติ, G20, เครือจักรภพแห่งชาติ, ANZUS, องค์การเพื่อความร่วมมือทางเศรษฐกิจและการพัฒนา (OECD), องค์การการค้าโลก, ความร่วมมือทางเศรษฐกิจเอเชียแปซิฟิกและหมู่เกาะแปซิฟิกฟอรั่ม
"""
question = "ชื่อทางการออสเตรเลีย"
question_answerer(question=question, context=context)

{'score': 0.8942838311195374,
 'start': 49,
 'end': 68,
 'answer': ' เครือรัฐออสเตรเลีย'}